In [1]:
import pandas as pd
import os
import sys
import importlib
import argparse
import logging
import sqlalchemy
import pymysql
pymysql.install_as_MySQLdb()
from datetime import datetime

In [2]:
source_mysql_host="172.16.0.227"
source_mysql_user="root"
source_mysql_password="123456789"
source_mysql_db="VN_REAL_CITY_2021_08"
source_mysql_table="REAL_CLEAN_KHANHHOA"

In [3]:
db_connection = sqlalchemy.create_engine("mysql://{}:{}@{}/{}?charset=utf8mb4".format(source_mysql_user,source_mysql_password,source_mysql_host,source_mysql_db),connect_args={'connect_timeout': 60000})
print("Connect database successfully") 

Connect database successfully


In [4]:
df = pd.read_sql('SELECT * FROM '+ source_mysql_table,con=db_connection)

In [5]:
df = df.drop_duplicates(subset=['ID_CLIENT', 'SITE','CREATED_DATE'], keep='last')

In [6]:
df.shape

(20652, 71)

In [7]:
df.head()

,ID_CLIENT,SITE,ADS_LINK,FOR_SALE,FOR_LEASE,TO_BUY,TO_LEASE,LAND_TYPE,ADS_DATE,PRICE,...,PROJECT_NAME,AGENCY_NAME,AGENCY_ADDRESS,AGENCY_CITY,AGENCY_TEL,AGENCY_WEBSITE,CHECK_CONVERT,PRO_FLAG,CREATED_DATE,DATE_ORIGINAL
0,1020640,nhadatcafeland,https://nhadat.cafeland.vn/can-tien-ban-gap-ch...,1,0,0,0,cc,2019-05-02,1.25,...,Dự án Khu đô thị Vĩnh Điềm Trung,None,None,None,None,None,0,12,2021-08-16,2021-08-16
1,1020843,nhadatcafeland,https://nhadat.cafeland.vn/ban-dat-xa-vinh-tha...,1,0,0,0,dn,2019-05-02,940.00,...,None,None,None,None,None,None,0,12,2021-08-16,2021-08-16
2,1020912,nhadatcafeland,https://nhadat.cafeland.vn/1cho-thue-can-ho-ng...,0,1,0,0,cc,2019-05-02,6.00,...,None,None,None,None,None,None,0,12,2021-08-21,2021-08-09
3,1020929,nhadatcafeland,https://nhadat.cafeland.vn/9-cho-thue-nha-nguy...,0,1,0,0,nmt,2019-05-02,12.00,...,None,None,None,None,None,None,0,12,2021-08-21,2021-08-09
4,1021100,nhadatcafeland,https://nhadat.cafeland.vn/ban-nhanh-lo-dat-th...,1,0,0,0,dn,2019-05-02,14.40,...,None,None,None,None,None,None,0,12,2021-08-16,2021-08-16


In [8]:
ls_att = ['FRONTAGE','ALLEY_ACCESS','PRO_WIDTH','PRO_LENGTH','SURFACE','PRICE']

In [9]:
def print_val(df,att,att_text):
    ls_id = list(df['ID_CLIENT'])
    for x in ls_id:
        d = df[df['ID_CLIENT'] == x]
        print(att,d.iloc[0][att])
        print(att_text,d.iloc[0][att_text])
        print(d.iloc[0]['ID_CLIENT'])
        print(d.iloc[0]['ADS_LINK'])
        print(d.iloc[0]['DETAILED_BRIEF'])
        print()

In [10]:
att = ls_att[5]
if (att == 'ALLEY_ACCESS'):
    att_text = 'ALLEY_TEXT'
else:
    att_text = att + '_TEXT'
print(att)
print(att_text)

PRICE
PRICE_TEXT


### Check crawling values

In [11]:
d1 = df[df[att] != 0] 

In [12]:
d1.shape

(18797, 71)

In [13]:
d1[att].describe()

count    1.879700e+04
mean     8.063108e+09
std      1.973074e+11
min      1.000000e-06
25%      8.000000e+02
50%      1.890000e+03
75%      5.000000e+03
max      9.765000e+12
Name: PRICE, dtype: float64

In [14]:
Q1 = d1[att].quantile(0.25)
Q3 = d1[att].quantile(0.75)
IQR = Q3 - Q1
sample1 = d1[(d1[att] < (Q1 - 3 * IQR)) | (d1[att] > (Q3 + 3 * IQR))]
sample1 = sample1[sample1[att] != sample1[att_text]]

In [15]:
sample1.shape

(855, 71)

In [16]:
sample1[['ID_CLIENT','SITE']].groupby(by=["SITE"]).count()

,ID_CLIENT
SITE,
alonhadat,374
batdongsan,4
batdongsan.com.vn,90
dothinet,27
homedy,16
nha.chotot.com,28
nhadatcafeland,45
rongbay,60
sosanhnha,80


In [17]:
sp1 = sample1.groupby("SITE").sample(frac=0.1, random_state=1)

In [18]:
sp1[['ID_CLIENT','SITE']].groupby(by=["SITE"]).count()

,ID_CLIENT
SITE,
alonhadat,37
batdongsan.com.vn,9
dothinet,3
homedy,2
nha.chotot.com,3
nhadatcafeland,4
rongbay,6
sosanhnha,8
tinbatdongsan,13


In [19]:
print_val(sp1,att,att_text)

PRICE 120000.0
PRICE_TEXT 30000.0
7110167
https://alonhadat.com.vn/can-ban-khach-san-moi-xay-xong-hem-hung-vuong-7110167.html
Cần bán khách sạn mới xây xong hẻm Hùng Vương  - Diện tích: 107m2 - Quy mô: 10 tầng , có hầm, 23 phòng khai thác và sân thượng mái che - sân thượng có hệ thống wc, nước, điện để làm thêm 2 phòng hoặc nhà hàng... - Phòng từ tầng 7 đến tầng 10 là view Biển. Tầng 1: lễ Tân và 1 phòng tầng 2-7: mỗi tầng 3 phòng (gồm 2 phòng giường đơn và 1 phòng đôi) Tầng 8-9: mỗi tầng 2 phòng (mỗi tầng 1 phòng đơn Vip và 1 đôi) Ghi chú: các phòng đôi từ tầng 2-9 đều bố trí làm được căn hộ. Hiện nay đã bố trí phòng đôi từ tầng 5-9 đều sử dụng như căn hộ Giá bán: 30 tỷ, có thương lượng với khách thiện chí.

PRICE 195000.0
PRICE_TEXT 0.0
8862335
https://alonhadat.com.vn/-dich-keo-dai-can-ban-gap-khach-san-8862335.html
Dịch kéo dài cần bán gấp khách sạn 100 phòng Nha Trang giá thấp nhất thị trường.   ️ 18 tầng với 100 phòng khách sạn tiêu chuẩn 3 sao đẳng cấp. ⚜️ Diện tích 260m2, ngang

PRICE 41000.0
PRICE_TEXT 40755.0
clap9XlJp
https://sosanhnha.com/248m-dat-tt-thanh-pho-nha-trang-41-ty-clap9XlJp
BÁN LÔ ĐẤT 247M2, ĐƯỜNG HỒNG BÀNG, TRUNG TÂM TP NHA TRANG. Lô đất mặt tiền đường Hồng Bàng, trung tâm tp. Nha Trang. Cách biển Trần Phú Nha Trang chỉ vài trăm mét. Đường Hồng Bàng là con đường lớn, trung tâm tp. Nha Trang. Là trung tâm khu bàn cờ kinh doanh sầm uất ngay trung tâm Nha Trang. Vị trí kinh doanh ưa thích của nhiều loại hình kinh doanh. Tập trung nhiều Khách sạn, Spa, văn phòng lớn nhỏ, nhà hàng ẩm thực và ăn uống thương hiệu. Chiều ngang rộng hơn 10m. Đi bộ chỉ vài phút đến ngay Trung Thương Mại Vincom, chợ trung tâm, siêu thị, trường học cấp 1,2,3… Vị trí lý tưởng thích hợp xây khách sạn, căn hộ mini, tòa nhà văn phòng…. Pháp lý sổ hồng đầy đủ, rõ ràng. Diện tích: 247.4m2 ( 10.6m x 23,3m ) Hướng đẹp: Đông Bắc Giá bán có thương lượng thiện chí: 165tr/m2 Liên hệ ngay để xem đất: 0982 041 941(Ms Tâm) Cám ơn đã đọc tin!

PRICE 152000.0
PRICE_TEXT 0.0
claqNZDD7
http

### Check splitting values

In [20]:
d2 = df[(df[att] == 0) & (df[att_text] != 0)]

In [21]:
d2.shape

(547, 71)

In [22]:
d2[att_text].describe()

count    5.470000e+02
mean     6.186270e+04
std      5.289000e+05
min      1.200000e+00
25%      7.500000e+02
50%      1.700000e+03
75%      4.510750e+03
max      1.080000e+07
Name: PRICE_TEXT, dtype: float64

In [23]:
Q1 = d2[att_text].quantile(0.25)
Q3 = d2[att_text].quantile(0.75)
IQR = Q3 - Q1
sample2 = d2[(d2[att_text] < (Q1 - 3 * IQR)) | (d2[att_text] > (Q3 + 3 * IQR))]
sample2 = sample2[sample2[att] != sample2[att_text]]

In [24]:
sample2.shape

(47, 71)

In [25]:
sample2[['ID_CLIENT','SITE']].groupby(by=["SITE"]).count()

,ID_CLIENT
SITE,
batdongsan.com.vn,7
diaoconline,1
dothinet,2
homedy,2
nhadatcafeland,10
rongbay,13
sosanhnha,5
tinbatdongsan,7


In [26]:
sp2 = sample2.groupby("SITE").sample(frac=1, random_state=1)

In [27]:
sp2[['ID_CLIENT','SITE']].groupby(by=["SITE"]).count()

,ID_CLIENT
SITE,
batdongsan.com.vn,7
diaoconline,1
dothinet,2
homedy,2
nhadatcafeland,10
rongbay,13
sosanhnha,5
tinbatdongsan,7


In [28]:
print_val(sp2,att,att_text)

PRICE 0.0
PRICE_TEXT 66752.4
30717808
https://batdongsan.com.vn/ban-dat-duong-dai-han-xa-cam-thanh-bac/sieu-pham-nen-lam-voi-6-6tr-m2-tong-gia-chi-700-800tr-so-do-tung-lo-lh-0962-919-715-pr30717808
Giá bán từ 6,6 triệu/m2 đất nền ven biển tại Cam Lâm - Khánh Hoà. Giá bán kèm chính sách đặc biệt chỉ có trong mùa dịch giãn cách xã hội. Tặng ngay 3 chỉ vàng cho 1 giao dịch thành công. Chiết khấu 2% khi mua từ 2 lô. LH: 0962 919 715. Vị trí: Thôn Tân Lập, xã Cam Thành Bắc, Cam Lâm, Khánh Hòa. Tổng diện tích cả KPL: 10.114 m2. Bảng hàng: 54 lô. Cơ cấu diện tích: - Liền kề 102m2 - 106m2 - 108m2 - 120m2. - SH mặt đường 123m2 - 133m2. Hình thức xây: Không yêu cầu xây. Thời gian sử dụng đất: Lâu dài. Pháp lý rõ ràng, sẵn sàng sổ đỏ, full thổ cư 100%. Liên kết vùng: - 50m tới cổng cụm công nghiệp Tân Lập QH 40ha. - 500m tới đường Đại Hàn QH 60m. - 2km tới Quốc Lộ 1A. - 10p tới cao tốc Vĩnh Hảo - Cam Lâm - Nha Trang. - 20p di chuyển tới sân bay quốc tế Cam Ranh. Gần trường mầm non, trường tiểu họ

### Total test cases

In [29]:
sample = pd.concat([sp1,sp2])

In [30]:
sample.shape

(132, 71)

In [31]:
sample = sample.drop_duplicates(keep='last')

In [32]:
sample.shape

(132, 71)